In [18]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci

In [13]:
from __future__ import absolute_import
from __future__ import print_function

import os
from shutil import copyfile

In [14]:
from testing_simulation import Simulation
from generator import TrafficGenerator
from model import TestModel
from visualization import Visualization
from utils import import_test_configuration, set_sumo, set_test_path


In [15]:
config = import_test_configuration(config_file='testing_settings.ini')
sumo_cmd = set_sumo(config['gui'], config['sumocfg_file_name'], config['max_steps'])
model_path, plot_path = set_test_path(config['models_path_name'], config['model_to_test'])

Model = TestModel(
    input_dim=config['num_states'],
    model_path=model_path
)

TrafficGen = TrafficGenerator(
    config['max_steps'], 
    config['n_cars_generated']
)

Visualization = Visualization(
    plot_path, 
    dpi=96
)
        
Simulation = Simulation(
    Model,
    TrafficGen,
    sumo_cmd,
    config['max_steps'],
    config['green_duration'],
    config['yellow_duration'],
    config['num_states'],
    config['num_actions']
)

In [16]:
TrafficGen.generate_routefile(seed = 10)

In [88]:
traci.start(sumo_cmd)

Could not connect to TraCI server at localhost:57299 [Errno 111] Connection refused
 Retrying in 1 seconds


(17, 'SUMO 0.32.0')

In [94]:
waiting_times = {}

In [113]:
l = traci.vehicle.getIDList()
len(l)

63

In [111]:
for car_id in l:
    wait_time = traci.vehicle.getAccumulatedWaitingTime(car_id)
    waiting_times[car_id] = wait_time

In [114]:
len(waiting_times)

68

In [107]:
sum(waiting_times.values())

1366.0

In [109]:
for i in range(0,100):
    traci.simulationStep()

In [115]:
traci.close()

In [6]:
print('\n----- Test episode')
simulation_time = Simulation.run(config['episode_seed'])  # run the simulation
print('Simulation time:', simulation_time, 's')

print("----- Testing info saved at:", plot_path)

copyfile(src='testing_settings.ini', dst=os.path.join(plot_path, 'testing_settings.ini'))

Visualization.save_data_and_plot(data=Simulation.reward_episode, filename='reward', xlabel='Action step', ylabel='Reward')
Visualization.save_data_and_plot(data=Simulation.queue_length_episode, filename='queue', xlabel='Step', ylabel='Queue lenght (vehicles)')


Could not connect to TraCI server at localhost:57213 [Errno 111] Connection refused
 Retrying in 1 seconds


(17, 'SUMO 0.32.0')